In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from neural_networks import Model
from neural_networks.preprocessing import one_hot_encoded

In [2]:
model = Model()

model.add_dense_layer(7, 4, activation='relu')
model.add_dense_layer(3, activation='softmax')

In [3]:
data = pd.read_csv('iris.csv')
data = shuffle(data.values)[:,1:]

In [4]:
split = 0.8
total = len(data)
split_index = int(total * split)

X, y = data[:,:-1], data[:,-1]
X_train, y_train = X[:split_index,].astype(float), y[:split_index,]
X_test, y_test = X[split_index:,].astype(float), y[split_index:,]

y_train = one_hot_encoded(y_train)
y_test = one_hot_encoded(y_test)

In [11]:
model.set_parameters(lr=0.1, loss='mse', optimization='sgd')

In [12]:
model.train(X_train, y_train, 50, batch_size=12)

Epoch  0
Loss:  0.09501601121552336
Accuracy:  0.65

Epoch  1
Loss:  0.07297052946892223
Accuracy:  0.6833333333333333

Epoch  2
Loss:  0.06731459474765816
Accuracy:  0.6666666666666666

Epoch  3
Loss:  0.0659419750119793
Accuracy:  0.65

Epoch  4
Loss:  0.06148382518375027
Accuracy:  0.8

Epoch  5
Loss:  0.05974254556101432
Accuracy:  0.8333333333333334

Epoch  6
Loss:  0.05834999076496873
Accuracy:  0.8416666666666667

Epoch  7
Loss:  0.05649107191863227
Accuracy:  0.8833333333333333

Epoch  8
Loss:  0.05501479922844161
Accuracy:  0.8916666666666667

Epoch  9
Loss:  0.05446480687470664
Accuracy:  0.875

Epoch  10
Loss:  0.05037246281972033
Accuracy:  0.9583333333333334

Epoch  11
Loss:  0.04956216056332157
Accuracy:  0.9416666666666667

Epoch  12
Loss:  0.04742500342872148
Accuracy:  0.95

Epoch  13
Loss:  0.050041504719253616
Accuracy:  0.875

Epoch  14
Loss:  0.04418377118095442
Accuracy:  0.9333333333333333

Epoch  15
Loss:  0.04789667400069726
Accuracy:  0.8666666666666667

Epoch

In [ ]:
model.set_parameters(lr=0.1, loss='mse', optimization='sgd')